In [4]:
import numpy as np
#from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from scm_irl.env.scm_irl_env import ScmIrlEnv
from sllib.conversions.geo_conversions import north_east_to_lat_lon, mps2knots, lat_lon_to_north_east
import numpy as np
import wandb
from wandb.integration.sb3 import WandbCallback
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import datetime
import os


import gymnasium as gym
# Check if the environment is registered

gym.register(
    id='ScmIrl-v0',
    entry_point='scm_irl.env:ScmIrlEnv',
    max_episode_steps=1000,
)

SEED = 42


date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")
model_name = f"scm_2a66ceaf61_{date_time}"

output_path = os.path.join("../outputs", model_name)
os.makedirs(output_path, exist_ok=True)


def make_env(env_id, rank, seed=0):
    def _init():
        path = "../data/raw/scenario_2a66ceaf61"
        env = ScmIrlEnv(path, mmsi=215811000, awareness_zone = [200, 500, 200, 200], start_time_reference=1577905000.0, render_mode="rgb_array")
        #env = gym.wrappers.RecordVideo(env, f"videos")  # record videos
        # if rank == 0:  # only add the RecordVideo wrapper for the first environment
        #     env = gym.wrappers.RecordVideo(env, f"{output_path}/videos")  # record videos
        #env = gym.wrappers.RecordEpisodeStatistics(env)  # record stats such as returns
        #env.seed(seed + rank)  # ensure each environment has a different seed
        return env
    return _init


config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 70000,
    "env_name": "ScmIrl-v0",
}

num_envs = 1
#env = make_env("ScmIrl-v0", rank=0, seed=SEED)
env = DummyVecEnv([make_env("ScmIrl-v0", i) for i in range(num_envs)])


def load_policy(policy_name, organization, env_name, venv):
    def policy_fn(obs, state, dones):
        actions = []
        states = []
        for env in venv.envs:
            # Get the current timestep
            timestep = env.timestep

            # Get the action from the vessel
            action = env.get_action_from_vessel(timestep)
            actions.append(action)
            states.append(None)

        #print(actions)
        return actions, states

    return policy_fn


expert = load_policy(
    "copy_action",
    organization="scm",
    env_name="ScmIrl-v0",
    venv=env,
)


from imitation.data import rollout

rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=1),
    rng=np.random.default_rng(SEED),
    exclude_infos = True,
    verbose = True,
)


/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment ScmIrl-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


mmsi: 215811000, start_time: 1577905000.0, end_time: 1577906100.0
Episode number 0 started
agent out of the scenario
Episode number 1 started


ValueError: expected one more observations than actions: 4 != 110 + 1

In [32]:
expert

<function __main__.load_policy.<locals>.policy_fn(obs, state, dones)>

In [2]:
rollouts

NameError: name 'rollouts' is not defined

In [6]:
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy

learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=SEED,
)
reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
)

In [8]:
learner

In [26]:
env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)

In [27]:
gail_trainer.train(200_000)

round:   0%|          | 0/12 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 500      |
|    gen/rollout/ep_rew_mean  | 34.4     |
|    gen/time/fps             | 1984     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 8        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.5      |
|    disc/disc_acc_expert             | 0        |
|    disc/disc_acc_gen                | 1        |
|    disc/disc_entropy                | 0.69     |
|    disc/disc_loss                   | 0.685    |
|    disc/disc_proportion_expert_pred | 0        |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   8%|▊         | 1/12 [00:17<03:17, 17.98s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 35.3        |
|    gen/rollout/ep_rew_wrapped_mean | 270         |
|    gen/time/fps                    | 2053        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 7           |
|    gen/time/total_timesteps        | 32768       |
|    gen/train/approx_kl             | 0.006985685 |
|    gen/train/clip_fraction         | 0.0338      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.687      |
|    gen/train/explained_variance    | 0.0555      |
|    gen/train/learning_rate         | 0.0004      |
|    gen/train/loss                  | 0.0315      |
|    gen/train/n_updates             | 5           |
|    gen/train/policy_gradient_loss  | -0.00158    |
|    gen/train/value_loss            | 4.7    

round:  17%|█▋        | 2/12 [00:35<02:54, 17.47s/it]

...

round:  83%|████████▎ | 10/12 [02:52<00:34, 17.26s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 89.1        |
|    gen/rollout/ep_rew_wrapped_mean | 261         |
|    gen/time/fps                    | 1963        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 8           |
|    gen/time/total_timesteps        | 180224      |
|    gen/train/approx_kl             | 0.008544098 |
|    gen/train/clip_fraction         | 0.0794      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.59       |
|    gen/train/explained_variance    | 0.961       |
|    gen/train/learning_rate         | 0.0004      |
|    gen/train/loss                  | -0.0294     |
|    gen/train/n_updates             | 50          |
|    gen/train/policy_gradient_loss  | -0.00521    |
|    gen/train/value_loss            | 0.0347 

round:  92%|█████████▏| 11/12 [03:10<00:17, 17.34s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 105          |
|    gen/rollout/ep_rew_wrapped_mean | 257          |
|    gen/time/fps                    | 1999         |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 8            |
|    gen/time/total_timesteps        | 196608       |
|    gen/train/approx_kl             | 0.0070858137 |
|    gen/train/clip_fraction         | 0.0632       |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.587       |
|    gen/train/explained_variance    | 0.971        |
|    gen/train/learning_rate         | 0.0004       |
|    gen/train/loss                  | 0.0119       |
|    gen/train/n_updates             | 55           |
|    gen/train/policy_gradient_loss  | -0.00405     |
|    gen/train/value_loss   

round: 100%|██████████| 12/12 [03:27<00:00, 17.32s/it]


In [28]:
env.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)

In [29]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 102.6 +/- 24.11514047232568
Rewards after training: 223.56 +/- 135.6097577610107
